In [48]:
%use lets-plot, krangl, kotlin-statistics, klaxon
import java.time.LocalDate
import java.util.*
import java.time.ZoneId

In [3]:
import java.net.URL
fun DataFrame.Companion.fromJsonObject(url: String, mapper: (JsonObject) -> JsonArray<*>): DataFrame {
    val uri = URL(url).toURI().toURL()
    val obj = Parser.default().parse(uri.openStream()) as JsonObject
    val arr = obj.let(mapper).toJsonString()
    return DataFrame.fromJsonString(arr)
}

fun <R> DataFrame.mapStringColumn(column: String, default: R, mapFunc: (String) -> R?) =
    addColumn(column) { it[column].map<String> { i -> mapFunc(i) ?: default } }
infix fun DataCol.notEq(i: Any) = !eq(i)

In [55]:
val dataUrl = "https://data.taipei/api/v1/dataset/42254d50-7eff-48bd-870f-5257fc4c1928?scope=resourceAquire"
val df = DataFrame.fromJsonObject(dataUrl) { it.obj("result")!!.array<Any?>("results")!! }
    .selectIf { it.name != "_id" }
    .mapStringColumn("百分比[%]", 0.0, String::toDoubleOrNull)
    .mapStringColumn("交易量[公斤]", 0, String::toIntOrNull)
    .rename("百分比[%]" to "百分比")
    .addColumn("年") {
            it["年別"].map<String> { y ->
                val year = y.substringBefore("年").toInt()
//                 LocalDate.of(year, 1, 1).atStartOfDay(ZoneId.systemDefault()).toInstant()
                year
            }
        }
df.schema()

DataFrame with 585 observations
年別       [Str]  64年, 64年, 64年, 64年, 64年, 64年, 64年, 64年, 64年, 64年, 64年, 64年, 64年, 65年, 65年, 65年, 65年, 65年, 65年, 65年, ...
交易量[公斤]  [Int]  221671801, 25564616, 4401414, 20171624, 54950853, 16268794, 31575089, 22813786, 17146579, 28779046, ...
百分比      [Dbl]  100, 11.53, 1.99, 9.1, 24.79, 7.34, 14.24, 10.29, 7.74, 12.98, 0, 0, 0, 100, 5.75, 2.64, 8.61, 34.32...
縣市別      [Str]  總計, 新北市, 新竹縣, 臺中市, 彰化縣, 南投縣, 雲林縣, 嘉義縣, 屏東縣, 臺北市, 宜蘭縣, 高雄市, 其他, 總計, 新北市, 新竹縣, 臺中市, 彰化縣, 南投縣, 雲林縣, 嘉義縣...
年        [Int]  64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, ...


In [107]:
val onlyTotal = lets_plot(df.filter{ it["縣市別"] eq "總計" }.toMap())
val df_withoutTotal = df.filter { it["縣市別"] notEq "總計" }
val withoutTotal = lets_plot(df_withoutTotal.toMap())

In [28]:
onlyTotal +
    geom_line { x = "年別"; y = "交易量[公斤]" } +
    ggtitle("歷年臺北市蔬菜交易總量")

In [29]:
withoutTotal +
    geom_bar() { x = "年別"; weight = "百分比"; y = "百分比";  fill = "縣市別" } +
    ggtitle("歷年臺北市蔬菜交易縣市比例")

In [30]:
withoutTotal + ggtitle("") +
    geom_point(size = 1.5) { x = "百分比"; y = "交易量[公斤]"; color = "縣市別" }

In [31]:
withoutTotal + ggtitle("") +
    geom_bar { x = "年別"; weight = "交易量[公斤]"; fill = "縣市別" }

In [32]:
withoutTotal + ggtitle("還沒想到") +
    geom_line() { x = "年別"; y = "交易量[公斤]"; color = "縣市別" }

In [91]:
val tp = df.filter{ it["縣市別"] eq "臺北市" }.run {
            val right = df.filter{ it["縣市別"] eq "總計" }
                .rename("交易量[公斤]" to "總交易量")
                .select("年別", "總交易量")
            innerJoin(right, "年別")
        }
    .addColumn("分界") { it["年"].le(92).map{ y -> "92年" + if(y) "前" else "後" } }

lets_plot(tp.toMap()) + ggtitle("臺北市交易量對總交易量之關係") +
    facet_grid(x = "分界") +
    geom_point() { x = "交易量[公斤]"; y = "總交易量"; color = "年" } +
    stat_smooth() { x = "交易量[公斤]"; y = "總交易量"; }

In [117]:
withoutTotal + ggtitle("各縣市交易比例分佈") +
    geom_boxplot() { y = "百分比"; x = "縣市別" }

In [123]:
lets_plot(data = df_withoutTotal
        .addColumn("分界") { it["年"].le(92).map{ y -> "92年" + if(y) "前" else "後" } }
        .toMap()) +
    ggtitle("各縣市交易比例分佈") +
    geom_boxplot() { y = "百分比"; x = "縣市別"; color = "分界" }